# Agenda: Week 2

1. Recap
2. Dtypes (the types of data that we can use in Pandas)
3. `NaN` ("not a number")
4. DataFrames (2D data structures that we use in Pandas)
5. Adding and removing data in our data frames
6. Useful methods in our data frames
7. Querying with boolean indexes
8. Querying with `loc`
9. Reading CSV data (meaning: Real-world data into our data frames)

# Last week, quick recap

1. Pandas has two types of data
    - series (1D)
    - data frame (2D)
2. We can create a series by passing a Python list (or a similar iterable, such as a NumPy array)
3. We can assign an index of our choosing to our series
    - By default, it numbers the elements starting at 0, just like a list/string/tuple
    - If we assign our own index, we can use integers, floats, or even strings
4. Retrieving
    - Retrieve via the index using `.loc`
    - Retrieve via the numeric position (starting at 0) with `.iloc`
    - We can use slices
5. Broadcasting
    - If we apply an operator to a series and a single value, that value is "broadcast" with the operator to every element of the series.  We get back a new series, the result of applying the value with the operator.
    - If we use broadcasting with `==` or other comparison operators, we get back a "boolean series," containing only `True` and `False` values
6. Boolean indexing
    - If we apply a boolean index to an existing series, then only those items that match a `True` value are returned.  Those matching a `False` value are ignored.
    - This is why we don't need to use `if` or `for` in Pandas.  We use boolean indexes to find all of the values that match what we want.

In [1]:
import numpy as np   # not critical, but might come in handy
import pandas as pd  # uses the standard alias "pd"
from pandas import Series, DataFrame  # this lets me use the names without pd. before them

In [3]:
s = Series([10, 20, 30, 20, 30, 40, 20, 30, 40, 50]) # default, numeric index
s

0    10
1    20
2    30
3    20
4    30
5    40
6    20
7    30
8    40
9    50
dtype: int64

In [4]:
s.mean()  # what is the mean of our series?

29.0

In [6]:
s.sum() / s.count()

29.0

In [7]:
# to retrieve an item, I can use either .loc or .iloc  (they'll be the same here)
s.loc[4]

30

In [8]:
s.iloc[8]

40

In [9]:
# If I want, I can define my own index on the series
s.index = list('abcdefghij')  # 10 elements require 10 index items

In [10]:
s

a    10
b    20
c    30
d    20
e    30
f    40
g    20
h    30
i    40
j    50
dtype: int64

In [11]:
s.loc['c']    # .loc uses the index

30

In [13]:
s.iloc[6]      # .iloc uses the position

20

In [14]:
s.head()   # show me the first 5 elements in s

a    10
b    20
c    30
d    20
e    30
dtype: int64

In [15]:
s.tail()   # show me the final 5 elements in s

f    40
g    20
h    30
i    40
j    50
dtype: int64

In [16]:
s.value_counts()  # how often is each value found in s?

20    3
30    3
40    2
10    1
50    1
dtype: int64

In [17]:
s.describe()   # gives me the descriptive statistics for s

count    10.00000
mean     29.00000
std      11.97219
min      10.00000
25%      20.00000
50%      30.00000
75%      37.50000
max      50.00000
dtype: float64

# Dtypes 

We see a Pandas series, and we think of it as a Python data structure. But really, we're seeing 1/2 of the actual data structure. Half of it exists in Python, and we use that half.  But the actual data is the other half, sort of like the submerged part of an iceberg, and it is implemented in the C language.  Why? Because C is super fast and efficient.

So the data isn't being stored in Python at all!  We don't get to use our familiar Python data structures, such as `int` or `float`.

Instead, we need to think like C programmers, at least a little bit.  There's no such thing as an "int" in C.  Rather, they have 8-bit ints, 16-bit ints, 32-bit ints, etc.  The more bits, the larger/smaller the ints are that we can handle.  But the more bits, the more memory each value takes up.

In Pandas, we think about these as "dtypes".

In [18]:
s = Series([10, 20, 30, 40, 50])  # Pandas sees only integers, so it guesses what dtype we want

In [19]:
s

0    10
1    20
2    30
3    40
4    50
dtype: int64

In [20]:
# The highest number we can get to with 64-bit ints is 2**32  (because of negative + positive numbers)
2**32

4294967296

# How to decide what dtype to use

- How big/small will the numbers be that you'll be dealing with?
- The bigger the dtype, the more numbers you can handle
- But the bigger the dtype, the more memory it'll use

If I have 1b integers in my series:
- 64-bit integers will result in 64 GB of memory to store this series
- 32-bit integers will need 32 GB
- 8-bit integers will need only 8GB

In [21]:
# how can I specify the dtype, if I don't like what Pandas is choosing by default?

s = Series([10, 20, 30, 40, 50], dtype=np.int32)  # notice: dtypes are from NumPy

In [22]:
s

0    10
1    20
2    30
3    40
4    50
dtype: int32

In [23]:
s = Series([10, 20, 30, 40, 50], dtype=np.int8) 

In [24]:
# 8-bit integers

s

0    10
1    20
2    30
3    40
4    50
dtype: int8

In [25]:
# what happens if I do this:

s * 10

0    100
1    -56
2     44
3   -112
4    -12
dtype: int8

In [26]:
s.loc[0] = 127
s

0    127
1     20
2     30
3     40
4     50
dtype: int8

In [27]:
s + 1

0   -128
1     21
2     31
3     41
4     51
dtype: int8

# What dtypes are available?

int, signed integers (both positive and negative)
- `np.int8`  (or `np.dtype('int8')`)
- `np.int16`
- `np.int32`
- `np.int64`

uint, unsigned integers (only positive)
- `np.uint8`
- `np.uint16`
- `np.uint32`
- `np.uint64`

Floating-point numbers
- `np.float16`
- `np.float32`
- `np.float64`
- `np.float128`

In [28]:
# how can I find out the dtype of a series?  Just ask it:

s.dtype

dtype('int8')

In [29]:
s

0    127
1     20
2     30
3     40
4     50
dtype: int8

In [30]:
# I now realize that I'm going to need a bigger dtype for my numbers in s.
# what can I do?

# option 1: assign to the dtype -- not possible in Pandas!
s.dtype = np.int16

AttributeError: can't set attribute 'dtype'

In [32]:
# option 2: Create a new series, based on the existing one, with a new dtype
# the way we do this is with the "astype" method

# just run "astype" on a series, passing as an argument the dtype you want
# this returns a new series -- it doesn't change s!
s.astype(np.int16)

0    127
1     20
2     30
3     40
4     50
dtype: int16

In [33]:
# this is how we really change the dtype
s = s.astype(np.int16)

In [34]:
s

0    127
1     20
2     30
3     40
4     50
dtype: int16

In [36]:
# Pandas automatically detects that it needs to use floating-point numbers
# because some of the input list's elements are floats

# So it chooses float64 -- not a bad choice!
s = Series([10, 20, 30.5, 40, 50.8])
s

0    10.0
1    20.0
2    30.5
3    40.0
4    50.8
dtype: float64

In [38]:
# I want to remove the floating-point parts from s
# this basically runs "int" on every element
s.astype(np.int64)

0    10
1    20
2    30
3    40
4    50
dtype: int64

# What happens if I create a series of strings?



In [39]:
# this created a series of *strings*
# the dtype here is ... object -- meaning, Python objects, handled by Python, not NumPy

s = Series('this is the most amazing course ever!'.split())
s

0       this
1         is
2        the
3       most
4    amazing
5     course
6      ever!
dtype: object

In [40]:
s = Series('10 20 30'.split())
s

0    10
1    20
2    30
dtype: object

In [41]:
s + s   # what will I get back?

0    1010
1    2020
2    3030
dtype: object

In [42]:
# get integers from our strings, using astype

s.astype(np.int8)

0    10
1    20
2    30
dtype: int8

In [45]:
# to use astype with object, use "object" not "np.object"
s.astype(object)

0    10
1    20
2    30
dtype: object

# Exercise: Calcuating from text input

1. Define a Python string containing integers, separated by spaces.
2. Define a Pandas series whose elements are strings, based on that string.
3. Calculate the mean of the numbers in the series.

In [46]:
text = '10 20 30 20 30 40 50 60'

s = Series(text.split())
s

0    10
1    20
2    30
3    20
4    30
5    40
6    50
7    60
dtype: object

In [53]:
# option 1: tell it the dtype at creation time
s = Series(text.split())
s

0    10
1    20
2    30
3    20
4    30
5    40
6    50
7    60
dtype: object

In [57]:
# option 2 (and better?) is to use astype, to convert from one dtype to another
s = s.astype(np.int32)
s

0    10
1    20
2    30
3    20
4    30
5    40
6    50
7    60
dtype: int32

In [58]:
s = Series([10, 20, 30.5, 40, 50])    # because there is a single float value, the dtype will be np.float64
s

0    10.0
1    20.0
2    30.5
3    40.0
4    50.0
dtype: float64

In [62]:
# what if I start with an int dtype?

s = Series([10, 20, 30, 40 ,50])  # int64 dtype, because that's the default
s.loc[3] = 40.5                   # set an element to be a float

# what's the dtype now?  float64 -- pandas changes the dtype, to accommodate what we want to add
s.dtype


dtype('float64')

In [63]:
s

0    10.0
1    20.0
2    30.0
3    40.5
4    50.0
dtype: float64

In [64]:
s.loc[1] = 'hello'  # what will happen to the dtype now?


In [65]:
s

0     10.0
1    hello
2     30.0
3     40.5
4     50.0
dtype: object

# NaN

`NaN` is a float, and is represents the lack of a number, or a number that you can (will) want to use.

In [70]:
# if we want to take the mean of five test scores, or four if they were absent for one test,
# then how do we record the fifth score if they were absent?

scores = Series([95, 89, 92, 86, 0])     # 0 is a bad idea
scores.mean()

72.4

In [71]:
# a better idea is to use np.nan (aka np.NaN)
# this value means "not a number" -- it cannot be calculated with

scores = Series([95, 89, 92, 86, np.nan]) 

In [72]:
scores

0    95.0
1    89.0
2    92.0
3    86.0
4     NaN
dtype: float64

In [74]:
scores.mean()  # will this work?  Yes -- by default, most methods in Pandas ignore NaN values

90.5

In [75]:
# let's ask the mean method what it does

help(scores.mean)    # notice -- I'm not running the method, so no () after its name

Help on method mean in module pandas.core.generic:

mean(axis: 'int | None | lib.NoDefault' = <no_default>, skipna=True, level=None, numeric_only=None, **kwargs) method of pandas.core.series.Series instance
    Return the mean of the values over the requested axis.
    
    Parameters
    ----------
    axis : {index (0)}
        Axis for the function to be applied on.
    skipna : bool, default True
        Exclude NA/null values when computing the result.
    level : int or level name, default None
        If the axis is a MultiIndex (hierarchical), count along a
        particular level, collapsing into a scalar.
    numeric_only : bool, default None
        Include only float, int, boolean columns. If None, will attempt to use
        everything, then use only numeric data. Not implemented for Series.
    **kwargs
        Additional keyword arguments to be passed to the function.
    
    Returns
    -------
    scalar or Series (if level specified)



In [76]:
# what if I want to be very strict, and keep all NaN values when I'm calculating the mean?
scores.mean(skipna=False)

nan

# No matter what you do, you have to deal with `NaN`

Nearly any data set you have will include `NaN`:
- Faulty measurements
- Lost records
- People didn't respond
- People didn't respond on time
- Things are being processed
- The data set was changed, and earlier records lack certain values

Some ways to deal with `NaN`:
1. Ignore it. If you invoke many methods (e.g., `mean`), then it'll just ignore `NaN`.
2. Remove it.  The `dropna` method returns a new series, just like the original, but without any `NaN` values.
3. Replace it. The `fillna` method returns a new series, just like the original, but with `NaN` values replaced by something else.  It's often a good idea to use the mean here, but you don't have to.

In [77]:
s = Series([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

s.loc[[2, 5, 7, 9]] = np.nan

In [78]:
s

0    10.0
1    20.0
2     NaN
3    40.0
4    50.0
5     NaN
6    70.0
7     NaN
8    90.0
9     NaN
dtype: float64

In [79]:
s.count()  # how many non-NaN values do we have?

6

In [80]:
s.describe()

count     6.000000
mean     46.666667
std      30.110906
min      10.000000
25%      25.000000
50%      45.000000
75%      65.000000
max      90.000000
dtype: float64

In [82]:
# to remove it, we can use s.dropna

s.dropna()  # returns a new series, with the index of s, and without s's NaN values

0    10.0
1    20.0
3    40.0
4    50.0
6    70.0
8    90.0
dtype: float64

In [83]:
# to change it, we can use s.fillna

s.fillna(3)

0    10.0
1    20.0
2     3.0
3    40.0
4    50.0
5     3.0
6    70.0
7     3.0
8    90.0
9     3.0
dtype: float64

In [84]:
# very common to use the mean for filling -- it's often less wrong than other values would be
s.fillna(s.mean())

0    10.000000
1    20.000000
2    46.666667
3    40.000000
4    50.000000
5    46.666667
6    70.000000
7    46.666667
8    90.000000
9    46.666667
dtype: float64

In [85]:
# if I run s.fillna(n), then we get back a new series, just like s,
# but in which all values of NaN have been replaced by n

s.fillna(999)

0     10.0
1     20.0
2    999.0
3     40.0
4     50.0
5    999.0
6     70.0
7    999.0
8     90.0
9    999.0
dtype: float64

# Exercise: Descriptive statistics with missing data

1. Create a series with 10 elements, whose values are the high temperatures forecast for the next 10 days where you live.
2. Assign `NaN` to several of these elements.
3. Replace `NaN` with the mean of the remaining elements.
4. After you replace `NaN` with the mean, has the mean shifted? 

In [86]:
s = Series([32, 33, 32, 32, 32, 32, 32, 33, 34, 35, 35, 36, 35, 35])
s

0     32
1     33
2     32
3     32
4     32
5     32
6     32
7     33
8     34
9     35
10    35
11    36
12    35
13    35
dtype: int64

In [87]:
# fancy indexing to set NaN on a bunch of elements
s.loc[[3, 7, 8, 12]] = np.nan
s

0     32.0
1     33.0
2     32.0
3      NaN
4     32.0
5     32.0
6     32.0
7      NaN
8      NaN
9     35.0
10    35.0
11    36.0
12     NaN
13    35.0
dtype: float64

In [89]:
s.mean()   # mean of the non-NaN elements

33.4

In [90]:
s.fillna(s.mean())   # replace the NaN values with the mean 

0     32.0
1     33.0
2     32.0
3     33.4
4     32.0
5     32.0
6     32.0
7     33.4
8     33.4
9     35.0
10    35.0
11    36.0
12    33.4
13    35.0
dtype: float64

In [91]:
s = s.fillna(s.mean())   # replace the s (with NaN values) with a new s (without NaN values)

In [93]:
s.mean()

33.4

# Next up:

- Data frames!
- Adding and removing data in our data frames

# Data frames

Data frames are 2-dimensional

- They have rows, with each row labeled -- with an index, the same index we have used for our series to date
- They have columns, with each column labeled -- with the same kind of index object, these are known as the "columns."

Each column in a data frame is a series object.  Meaning: Each column can have its own distinct dtype.

In [94]:
# let's create a simple data frame, using a list of lists

df = DataFrame([[10, 20, 30, 40],
                [50, 60, 70, 80],
                [90, 100, 110, 120]])
df

,0,1,2,3
0,10,20,30,40
1,50,60,70,80
2,90,100,110,120


In [98]:
# let's create our data frame again, labeling the rows and the columns

# here, we'll use a list of lists, with each inner list representing
# one row in the data frame

df = DataFrame([[10, 20, 30, 40],
                [50, 60, 70, 80],
                [90, 100, 110, 120]],
              index=list('xyz'),
              columns=list('abcd'))
df

,a,b,c,d
x,10,20,30,40
y,50,60,70,80
z,90,100,110,120


In [99]:
# get the index from df
df.index

Index(['x', 'y', 'z'], dtype='object')

In [100]:
# get the columns from df
df.columns

Index(['a', 'b', 'c', 'd'], dtype='object')

In [101]:
# to retrieve a column, use [] and the column name

df['a']   # get column a -- we get a series, whose index is df's index

x    10
y    50
z    90
Name: a, dtype: int64

In [102]:
df['c']  # get column c -- we get a series

x     30
y     70
z    110
Name: c, dtype: int64

In [104]:
# if I want two columns, I can use fancy indexing
df[['a', 'c']] # I get a new data frame back, with the index and the columns I specified

,a,c
x,10,30
y,50,70
z,90,110


In [106]:
# remember I said that you should use .loc/.iloc to retrieve via the index on a series?
# that's because we use .loc/.iloc to retrieve rows from our data frame

# we get a series, because we're asked for 1D data, and that's how Pandas shows us 1D data

df.loc['y']  # retrieve the row at index y

a    50
b    60
c    70
d    80
Name: y, dtype: int64

In [109]:
df.loc[['x', 'y']]   # fancy indexing to retrieve two rows

,a,b,c,d
x,10,20,30,40
y,50,60,70,80


In [108]:
df.loc['x':'y']  # use a slice to retrieve two rows

,a,b,c,d
x,10,20,30,40
y,50,60,70,80


In [110]:
# if I want to retrieve from a particular row, and a particular column, I use .loc

# I want row y, column c

df.loc['y', 'c']  

70

In [111]:
df.loc['y', 'c'] = 999

In [112]:
df

,a,b,c,d
x,10,20,30,40
y,50,60,999,80
z,90,100,110,120


In [113]:
# don't be tempted to use this syntax:
# this will work sometimes, but NOT ALL OF THE TIME, and you'll get a warning,
# and it's very unpleasant to have code that works sometimes, but not always

# if you have ] just before [ , you are almost certainly doing something wrong

df.loc['y']['c'] = 888

In [114]:
df

,a,b,c,d
x,10,20,30,40
y,50,60,888,80
z,90,100,110,120


# Exercise: Mini-market

1. Define a data frame in which we have three columns: `product_name` , `price` and `sales`.  Define the data frame such that we have five different products.  The index will be numeric.
2. Retrieve the `product_name` column.
3. Retrieve the entire row for one product.
4. Find the average price of items in the store.
5. Find me the greatest number of sales that any product has.
6. Choose a product, and get its price.
7. Choose a product, and get the number of sales.

In [134]:
df = DataFrame([['apple', 1, 20],
                ['banana', 1.2, 15],
                ['cucumber', 0.5, 30],
                ['dill', 2, 10],
                ['endive', 5, 3]],
              columns=['product_name', 'price', 'sales'])
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [117]:
df.dtypes   # show me the dtype for each column in the system

product_name     object
price           float64
sales             int64
dtype: object

In [118]:
df['product_name']  # [] alone are for columns

0       apple
1      banana
2    cucumber
3        dill
4      endive
Name: product_name, dtype: object

In [119]:
# shh  don't tell this secret
# you can also say
df.product_name

0       apple
1      banana
2    cucumber
3        dill
4      endive
Name: product_name, dtype: object

In [120]:
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [121]:
df.loc[2] # get cucumber info

product_name    cucumber
price                0.5
sales                 30
Name: 2, dtype: object

In [122]:
df['price'].mean()

1.94

In [124]:
df['sales'].max()   # what is the largest number in df['sales']?

30

In [125]:
# Using loc to retrieve row 3, column 'price'
df.loc[3, 'price']

2.0

In [126]:
# Using loc to retrieve row 3, column 'sales'
df.loc[3, 'sales']

10

# Adding rows

I can add a new row to the data frame by ... assigning to a (new) index.  If I assign to an index that already exists, I'll replace its contents.

In [127]:
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [135]:
df.loc[5] = ['fennel', 1, 100]  # we must provide a value for each column
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3
5,fennel,1.0,100


In [136]:
# adding a new column -- we do this by assigning to a new (unused) column name

df['newcol'] = [10, 20, 30, 40, 50, 60]  # we must have the right number of elements

In [132]:
df

,product_name,price,sales,newcol
0,apple,1.0,20,10
1,banana,1.2,15,20
2,cucumber,0.5,30,30
3,dill,2.0,10,40
4,endive,5.0,3,50
5,fennel,1.0,100,60


In [137]:
# remember that if we want, we can perform mathematical operations on two different series
# the operation will be matched up by the index

# so what happens if I multiply df['price'] (a series) with df['sales'] (another series)
# with the same index?

df['price'] * df['sales']

0     20.0
1     18.0
2     15.0
3     20.0
4     15.0
5    100.0
dtype: float64

In [138]:
# now we can find out how much we earned per item
df['revenue'] = df['price'] * df['sales']

In [139]:
df

,product_name,price,sales,newcol,revenue
0,apple,1.0,20,10,20.0
1,banana,1.2,15,20,18.0
2,cucumber,0.5,30,30,15.0
3,dill,2.0,10,40,20.0
4,endive,5.0,3,50,15.0
5,fennel,1.0,100,60,100.0


In [141]:
# what if I want to remove a row?
df.drop(4)  # I get back a new data frame, based on df, without the "endive row"

,product_name,price,sales,newcol,revenue
0,apple,1.0,20,10,20.0
1,banana,1.2,15,20,18.0
2,cucumber,0.5,30,30,15.0
3,dill,2.0,10,40,20.0
5,fennel,1.0,100,60,100.0


In [142]:
# notice, it's still there!
df

,product_name,price,sales,newcol,revenue
0,apple,1.0,20,10,20.0
1,banana,1.2,15,20,18.0
2,cucumber,0.5,30,30,15.0
3,dill,2.0,10,40,20.0
4,endive,5.0,3,50,15.0
5,fennel,1.0,100,60,100.0


In [143]:
# pass a list of indexes, and we'll remove all of them
df.drop([1, 
        ])   # we don't specify axis='rows', so it uses rows by default

,product_name,price,sales,newcol,revenue
0,apple,1.0,20,10,20.0
2,cucumber,0.5,30,30,15.0
3,dill,2.0,10,40,20.0
5,fennel,1.0,100,60,100.0


In [144]:
# what about removing columns?
# there, we also use "drop" -- but we need to tell pandas that we're dropping a column

df.drop('newcol', axis='columns')  # axis='columns'

,product_name,price,sales,revenue
0,apple,1.0,20,20.0
1,banana,1.2,15,18.0
2,cucumber,0.5,30,15.0
3,dill,2.0,10,20.0
4,endive,5.0,3,15.0
5,fennel,1.0,100,100.0


In [145]:
df = df.drop('newcol', axis='columns')
df

,product_name,price,sales,revenue
0,apple,1.0,20,20.0
1,banana,1.2,15,18.0
2,cucumber,0.5,30,15.0
3,dill,2.0,10,20.0
4,endive,5.0,3,15.0
5,fennel,1.0,100,100.0


# Exercise: Family and friends

1. Define a new data frame in which we have 4 rows and 3 columns. The columns will be `first_name`, `last_name`, and `age`.  The rows will represent several people you know and/or are related to and/or want to be related to.
2. Add a new row, for an additional person.
3. Add a new column, `shoesize`, for all people.
4. Calculate the descriptive statistics for both `age` and `shoesize`.

In [147]:
df = DataFrame([['Reuven', 'Lerner', 52],
                ['Atara', 'Lerner-Friedman', 21],
               ['Shikma', 'Lerner-Friedman', 19],
               ['Amotz', 'Lerner-Friedman', 16]],
              columns=['first_name', 'last_name', 'age'])
df

,first_name,last_name,age
0,Reuven,Lerner,52
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16


In [148]:
df[['first_name', 'last_name']]

,first_name,last_name
0,Reuven,Lerner
1,Atara,Lerner-Friedman
2,Shikma,Lerner-Friedman
3,Amotz,Lerner-Friedman


In [149]:
df[['first_name', 'age']]

,first_name,age
0,Reuven,52
1,Atara,21
2,Shikma,19
3,Amotz,16


In [151]:
df = DataFrame([['Reuven', 'Lerner', 52],
                ['Atara', 'Lerner-Friedman', 21],
               ['Shikma', 'Lerner-Friedman', 19],
               ['Amotz', 'Lerner-Friedman', 16]],
              columns=['first_name', 'last_name', 'age'])
df

,first_name,last_name,age
0,Reuven,Lerner,52
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16


In [152]:
df.loc[4] = ['Random', 'Person', 30]
df

,first_name,last_name,age
0,Reuven,Lerner,52
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16
4,Random,Person,30


In [153]:
# add a new column: shoesize

df['shoesize'] = [46, 42, 42, 44, 40]
df

,first_name,last_name,age,shoesize
0,Reuven,Lerner,52,46
1,Atara,Lerner-Friedman,21,42
2,Shikma,Lerner-Friedman,19,42
3,Amotz,Lerner-Friedman,16,44
4,Random,Person,30,40


In [154]:
df['age'].describe()

count     5.000000
mean     27.600000
std      14.604794
min      16.000000
25%      19.000000
50%      21.000000
75%      30.000000
max      52.000000
Name: age, dtype: float64

In [155]:
df['shoesize'].describe()

count     5.000000
mean     42.800000
std       2.280351
min      40.000000
25%      42.000000
50%      42.000000
75%      44.000000
max      46.000000
Name: shoesize, dtype: float64

In [157]:
# I can run .describe on a data frame, not just on a single series
# if I do that, I get results for *EACH* column in the data frame

# fancy indexing
df[['age', 'shoesize']].describe()

,age,shoesize
count,5.000000,5.000000
mean,27.600000,42.800000
std,14.604794,2.280351
min,16.000000,40.000000
25%,19.000000,42.000000
50%,21.000000,42.000000
75%,30.000000,44.000000
max,52.000000,46.000000


# Next up

1. Setting/resetting the index
2. Useful methods/accessors we can use on a data frame
3. Use boolean indexes to query our data frame

In [158]:
df

,first_name,last_name,age,shoesize
0,Reuven,Lerner,52,46
1,Atara,Lerner-Friedman,21,42
2,Shikma,Lerner-Friedman,19,42
3,Amotz,Lerner-Friedman,16,44
4,Random,Person,30,40


In [159]:
df = DataFrame([['apple', 1, 20],
                ['banana', 1.2, 15],
                ['cucumber', 0.5, 30],
                ['dill', 2, 10],
                ['endive', 5, 3]],
              columns=['product_name', 'price', 'sales'])
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [160]:
# wouldn't it make more sense to have product_name be the index, rather than some numbers?
# the set_index method returns a new data frame in which the specified column is the index

df.set_index('product_name')

,price,sales
product_name,,
apple,1.0,20
banana,1.2,15
cucumber,0.5,30
dill,2.0,10
endive,5.0,3


In [161]:
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [162]:
# to actually change the index, we need to assign the result of set_index to df

df = df.set_index('product_name')

In [163]:
df

,price,sales
product_name,,
apple,1.0,20
banana,1.2,15
cucumber,0.5,30
dill,2.0,10
endive,5.0,3


In [164]:
# product_name is no longer available as a column (by its name)

df['product_name']

KeyError: 'product_name'

In [165]:
df

,price,sales
product_name,,
apple,1.0,20
banana,1.2,15
cucumber,0.5,30
dill,2.0,10
endive,5.0,3


In [166]:
df.loc['dill']

price     2.0
sales    10.0
Name: dill, dtype: float64

In [168]:
df.loc['apple']

price     1.0
sales    20.0
Name: apple, dtype: float64

In [169]:
# fancy index
df.loc[['apple', 'dill']]

,price,sales
product_name,,
apple,1.0,20
dill,2.0,10


In [170]:
# if I want to make the index back into a regular column, I can run reset_index
df.reset_index()

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [171]:
df

,price,sales
product_name,,
apple,1.0,20
banana,1.2,15
cucumber,0.5,30
dill,2.0,10
endive,5.0,3


In [172]:
# to really change df, such that product_name is back as a column and not as an index, I have 
# to assign it back to df

df = df.reset_index()

In [173]:
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [174]:
df = df.set_index('product_name')
df

,price,sales
product_name,,
apple,1.0,20
banana,1.2,15
cucumber,0.5,30
dill,2.0,10
endive,5.0,3


In [175]:
# retrieve the index, and we get the product names!
df.index

Index(['apple', 'banana', 'cucumber', 'dill', 'endive'], dtype='object', name='product_name')

In [177]:
df.index[0]

'apple'

In [178]:
df = df.reset_index()

In [179]:
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


# Useful (series) methods on data frames

The general rule of thumb is: If you can run a method on a series, then you can also run it on a data frame.

- If, when you run the method on a series, you get back a single (scalar) value, then running it on a data frame will give you a series back, one value for each column. (The data frame's columns will be the index of the returned series)
- If, when you run the method on a series, you get back a series, then running it on a data frame will give you a data frame back, one series for each column. The data frame's columns will be the columns of the returned data frame.

In [180]:
df

,product_name,price,sales
0,apple,1.0,20
1,banana,1.2,15
2,cucumber,0.5,30
3,dill,2.0,10
4,endive,5.0,3


In [181]:
df.describe()   # describe returns a series -- so we'll get a data frame back, with 1 col for each of df's columns

,price,sales
count,5.000000,5.000000
mean,1.940000,15.600000
std,1.793878,10.212737
min,0.500000,3.000000
25%,1.000000,10.000000
50%,1.200000,15.000000
75%,2.000000,20.000000
max,5.000000,30.000000


In [182]:
df['revenue'] = df['price'] * df['sales']

In [183]:
df

,product_name,price,sales,revenue
0,apple,1.0,20,20.0
1,banana,1.2,15,18.0
2,cucumber,0.5,30,15.0
3,dill,2.0,10,20.0
4,endive,5.0,3,15.0


In [184]:
df.describe()

,price,sales,revenue
count,5.000000,5.000000,5.00000
mean,1.940000,15.600000,17.60000
std,1.793878,10.212737,2.50998
min,0.500000,3.000000,15.00000
25%,1.000000,10.000000,15.00000
50%,1.200000,15.000000,18.00000
75%,2.000000,20.000000,20.00000
max,5.000000,30.000000,20.00000


In [186]:
# what about our individual aggregate methods, such as mean, or std?

df[['price', 'sales', 'revenue']].mean()  # on a series, mean returns a single float value. So on a data frame, we'll get back a series of floats

price       1.94
sales      15.60
revenue    17.60
dtype: float64

In [187]:
df.shape  # this returns the number of rows and columns as a 2-element tuple

(5, 4)

In [188]:
df

,product_name,price,sales,revenue
0,apple,1.0,20,20.0
1,banana,1.2,15,18.0
2,cucumber,0.5,30,15.0
3,dill,2.0,10,20.0
4,endive,5.0,3,15.0


In [190]:
df.head(2)   # show me the first 2 rows in the data frame

,product_name,price,sales,revenue
0,apple,1.0,20,20.0
1,banana,1.2,15,18.0


In [191]:
df.tail(2)  # show me the final 2 rows in the data frame

,product_name,price,sales,revenue
3,dill,2.0,10,20.0
4,endive,5.0,3,15.0


In [192]:
df.dtypes

product_name     object
price           float64
sales             int64
revenue         float64
dtype: object

In [193]:
df

,product_name,price,sales,revenue
0,apple,1.0,20,20.0
1,banana,1.2,15,18.0
2,cucumber,0.5,30,15.0
3,dill,2.0,10,20.0
4,endive,5.0,3,15.0


In [194]:
# let's add some NaN values!

df.loc[2, 'sales'] = np.nan
df.loc[1, 'revenue'] = np.nan
df.loc[4, 'price'] = np.nan



In [195]:
df

,product_name,price,sales,revenue
0,apple,1.0,20.0,20.0
1,banana,1.2,15.0,NaN
2,cucumber,0.5,NaN,15.0
3,dill,2.0,10.0,20.0
4,endive,NaN,3.0,15.0


In [196]:
df.dtypes

product_name     object
price           float64
sales           float64
revenue         float64
dtype: object

In [197]:
# remember how we can call dropna on a series, to remove any NaN values?
# we can also call it on a data frame.  Any row containing NaN will be eliminated.

df.dropna()

,product_name,price,sales,revenue
0,apple,1.0,20.0,20.0
3,dill,2.0,10.0,20.0


In [198]:
df

,product_name,price,sales,revenue
0,apple,1.0,20.0,20.0
1,banana,1.2,15.0,NaN
2,cucumber,0.5,NaN,15.0
3,dill,2.0,10.0,20.0
4,endive,NaN,3.0,15.0


In [199]:
# what if I want to use fillna?

# If I call df.fillna() and provide a scalar value, all NaNs will be filled with that value
df.fillna(999)

,product_name,price,sales,revenue
0,apple,1.0,20.0,20.0
1,banana,1.2,15.0,999.0
2,cucumber,0.5,999.0,15.0
3,dill,2.0,10.0,20.0
4,endive,999.0,3.0,15.0


In [201]:
# if I call df.mean(), I get the mean of each column (as a series)
df.mean()

/var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_10226/1166147312.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


price       1.175
sales      12.000
revenue    17.500
dtype: float64

In [202]:
# df.fillna, if you give it a *series*, will fill in each column's NaN values
# based on the index in df.mean

df.fillna(df.mean())

/var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_10226/1263820274.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.fillna(df.mean())


,product_name,price,sales,revenue
0,apple,1.000,20.0,20.0
1,banana,1.200,15.0,17.5
2,cucumber,0.500,12.0,15.0
3,dill,2.000,10.0,20.0
4,endive,1.175,3.0,15.0


# Exercise: High and low temperatures

1. Create a data frame in which the index will be strings, the `MMDD` strings representing months and days.  The two columns will be the forecast high temperature for that day, and the forecast low temperature for that day.  So the data frame will have an index and two numeric columns.
2. Calculate descriptive statistics for both columns.
3. Assign 5 `NaN` values to the numeric columns.
4. Fill in the `NaN` values with the mean from each column.

In [205]:
df = DataFrame([[32, 23],
                [33, 21], [33, 22], [32, 23],
               [32, 22], [32, 22], [32, 22], [32, 22], [35, 23], [36, 24]],
              columns=['high', 'low'],
              index='0809 0810 0811 0812 0813 0814 0815 0816 0817 0818'.split())
df

,high,low
0809,32,23
0810,33,21
0811,33,22
0812,32,23
0813,32,22
0814,32,22
0815,32,22
0816,32,22
0817,35,23
0818,36,24


In [206]:
df = df.reset_index()

In [207]:
df

,index,high,low
0,0809,32,23
1,0810,33,21
2,0811,33,22
3,0812,32,23
4,0813,32,22
5,0814,32,22
6,0815,32,22
7,0816,32,22
8,0817,35,23
9,0818,36,24


In [208]:
df = df.set_index('index')
df

,high,low
index,,
0809,32,23
0810,33,21
0811,33,22
0812,32,23
0813,32,22
0814,32,22
0815,32,22
0816,32,22
0817,35,23


In [209]:
df.dtypes

high    int64
low     int64
dtype: object

In [210]:
df.describe()  

,high,low
count,10.000000,10.000000
mean,32.900000,22.400000
std,1.449138,0.843274
min,32.000000,21.000000
25%,32.000000,22.000000
50%,32.000000,22.000000
75%,33.000000,23.000000
max,36.000000,24.000000


In [211]:
df

,high,low
index,,
0809,32,23
0810,33,21
0811,33,22
0812,32,23
0813,32,22
0814,32,22
0815,32,22
0816,32,22
0817,35,23


In [212]:
df.loc['0810', 'high'] = np.nan
df.loc['0810', 'low'] = np.nan
df.loc['0813', 'high'] = np.nan
df.loc['0815', 'low'] = np.nan
df.loc['0817', 'low'] = np.nan
df.loc['0818', 'high'] = np.nan
df

,high,low
index,,
0809,32.0,23.0
0810,NaN,NaN
0811,33.0,22.0
0812,32.0,23.0
0813,NaN,22.0
0814,32.0,22.0
0815,32.0,NaN
0816,32.0,22.0
0817,35.0,NaN


In [213]:
df.describe()

,high,low
count,7.000000,7.000000
mean,32.571429,22.571429
std,1.133893,0.786796
min,32.000000,22.000000
25%,32.000000,22.000000
50%,32.000000,22.000000
75%,32.500000,23.000000
max,35.000000,24.000000


In [214]:
df.dropna()

,high,low
index,,
0809,32.0,23.0
0811,33.0,22.0
0812,32.0,23.0
0814,32.0,22.0
0816,32.0,22.0


In [216]:
# we can give df.dropna a "thresh" argument, indicating how many non-NaN values 
# a row can have to not be eliminated.  

df.dropna(thresh=1)  # this means: if you have 1 good value, the row can stay

,high,low
index,,
0809,32.0,23.0
0811,33.0,22.0
0812,32.0,23.0
0813,NaN,22.0
0814,32.0,22.0
0815,32.0,NaN
0816,32.0,22.0
0817,35.0,NaN
0818,NaN,24.0


In [217]:
df.fillna(df.mean())

,high,low
index,,
0809,32.000000,23.000000
0810,32.571429,22.571429
0811,33.000000,22.000000
0812,32.000000,23.000000
0813,32.571429,22.000000
0814,32.000000,22.000000
0815,32.000000,22.571429
0816,32.000000,22.000000
0817,35.000000,22.571429


# Boolean indexes

Remember: If we broadcast a comparison operator to a series, then we get back a new series, with the original's index, of `True` and `False` values.  We can then apply that to a series in `[]`, getting only those values back where the booleans were `True`.

In [219]:
# show me elements of df['high'] where the high is > mean
df['high'][df['high'] > df['high'].mean()]

index
0811    33.0
0817    35.0
Name: high, dtype: float64

In [220]:
# show me EVERY COLUMN where df['high'] is > mean
# this will apply our boolean series as a mask to every row, returning only those where it's True
df[df['high'] > df['high'].mean()]

,high,low
index,,
0811,33.0,22.0
0817,35.0,NaN


# Exercise: Mini market

1. Create (or re-create) the data frame for the mini-market.  The index should be the product names, and the columns should be `price` and `sales`.
2. Find those products whose revenue is greater than the mean.  (Yes, you'll need to calculate revenue.)
3. Find those products whose sales are less than the mean.  
4. Find those products whose sales are greater than the mean, and revenue is also greater than the mean. (Remember that we can combine conditions with `&` and `|`.)

In [222]:
df = DataFrame([['apple', 1, 20],
                ['banana', 1.2, 15],
                ['cucumber', 0.5, 30],
                ['dill', 2, 10],
                ['endive', 5, 3]],
              columns=['product_name', 'price', 'sales']).set_index('product_name')

df



,price,sales
product_name,,
apple,1.0,20
banana,1.2,15
cucumber,0.5,30
dill,2.0,10
endive,5.0,3


In [223]:
# what products have revenue > mean? 
df['revenue'] = df['price'] * df['sales']
df

,price,sales,revenue
product_name,,,
apple,1.0,20,20.0
banana,1.2,15,18.0
cucumber,0.5,30,15.0
dill,2.0,10,20.0
endive,5.0,3,15.0


In [225]:
# apply this boolean series to the entire data frame
# and get back all into about the products with above-average revenue

df[df['revenue'] > df['revenue'].mean()]

,price,sales,revenue
product_name,,,
apple,1.0,20,20.0
banana,1.2,15,18.0
dill,2.0,10,20.0


In [227]:
# Find those products whose sales are less than the mean.  

df[df['sales'] < df['sales'].mean()]

,price,sales,revenue
product_name,,,
banana,1.2,15,18.0
dill,2.0,10,20.0
endive,5.0,3,15.0


In [230]:
# Find those products whose sales are greater than the mean, 
# and revenue is also greater than the mean. (Remember that we can combine conditions with & and |.)

df[(df['sales'] > df['sales'].mean()) & 
   (df['revenue'] > df['revenue'].mean())]

,price,sales,revenue
product_name,,,
apple,1.0,20,20.0


# Next up

1. Using `.loc` to retrieve data in a complex way
2. Reading CSV files!

In [231]:
df

,price,sales,revenue
product_name,,,
apple,1.0,20,20.0
banana,1.2,15,18.0
cucumber,0.5,30,15.0
dill,2.0,10,20.0
endive,5.0,3,15.0


In [232]:
# we've seen that we can apply a boolean series to a data frame, and get back
# only those rows where the series is True.

# we can do better than that, though
# if I use .loc with a boolean series, I can then choose columns, as well

# the basic way to think about .loc is:

# df.loc[row_selector, column_selector]

In [236]:
df.loc[['dill', 'apple'],      # row selector       
       ['sales', 'revenue']]   # column selector

,sales,revenue
product_name,,
dill,10,20.0
apple,20,20.0


In [235]:
df.loc[df['price'] > df['price'].mean(),     # row selector -- boolean series finding high-priced items
       ['sales', 'revenue']]                 # column selector -- list of columns I want

,sales,revenue
product_name,,
dill,10,20.0
endive,3,15.0


# Exercise: Family and friends

1. (Re-)create your data frame containing family and friends information.  The columns will be `name`, `age`, and `shoesize`.
2. Make the `name` column into the index.
3. Find the shoe size of people whose `age` is above average.
4. Find the average age of people whose shoe size is below average.


In [238]:
df = DataFrame([['Reuven',  52, 46],
                ['Atara',  21, 42],
               ['Shikma',  19, 42],
               ['Amotz',  16, 44]],
              columns=['name', 'age', 'shoesize']).set_index('name')
df

,age,shoesize
name,,
Reuven,52,46
Atara,21,42
Shikma,19,42
Amotz,16,44


In [239]:
# find the shoe size of people whose age is above average

df.loc[
  df['age'] > df['age'].mean(),  # row selector: age above average
  'shoesize']                    # column selector: shoesize

name
Reuven    46
Name: shoesize, dtype: int64

In [242]:
# Find the average age of people whose shoe size is below average.

df.loc[
 df['shoesize']  < df['shoesize'].mean(),   # row selector: shoe size is below average
 'age'                                      # column selector: age
].mean()   # get these people's average age

20.0

In [244]:
# Find the average age and shoe size of people whose shoe size is below average.

df.loc[
 df['shoesize']  < df['shoesize'].mean(),   # row selector: shoe size is below average
 ['age', 'shoesize']                        # column selector: age and shoesize
].mean()   # get these people's average age

age         20.0
shoesize    42.0
dtype: float64

In [245]:
# a huge amount of data is stored in CSV files (i.e., comma-separated values)

# CSV is a text format
# - every line of the file is a record
# - fields in each record are separated by commas (thus, comma-separated values)

In [246]:
# in Jupyter, I can execute Unix commands with ! at the start of a line
# the "head" command in Unix shows the first 10 lines of a file

!head nyc_taxi_2019-01.csv